# Use Qiskit’s VQE implementation to solve the Max-Cut problem for a graph of 5 vertices in which the connections are $(0, 1)$, $(1, 2)$, $(2, 3)$, $(3, 4)$ and $(4, 0)$.

Para resolver este problema utilizando VQE necesitamos expresar el hamiltoniano como una combinación lineal de hamiltonianos de pauli.

In [2]:
from qiskit.quantum_info import SparsePauliOp
 
operator = SparsePauliOp(["ZZIII", "IZZII",'IIZZI','IIIZZ','ZIIIZ'], coeffs = [1,1,1,1,1])
 
# equivalent to:
print(repr(operator))

SparsePauliOp(['ZZIII', 'IZZII', 'IIZZI', 'IIIZZ', 'ZIIIZ'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


In [3]:
for pauli, coeff in sorted(operator.label_iter()):
    print(f"{coeff.real:+.8f} * {pauli}")

+1.00000000 * IIIZZ
+1.00000000 * IIZZI
+1.00000000 * IZZII
+1.00000000 * ZIIIZ
+1.00000000 * ZZIII


In [6]:
import numpy as np
from qiskit_algorithms import VQE
from qiskit_aer import AerSimulator
from qiskit.primitives import Sampler
#from qiskit.utils import algorithm_globals
import numpy as np
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import EfficientSU2
# define Aer Estimator for noiseless statevector simulation


seed = 1235
algorithm_globals.random_seed = seed

ansatz = EfficientSU2(num_qubits=5, reps=1, entanglement="linear",
insert_barriers = True)

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)

optimizer = COBYLA()

initial_point = np.random.random(ansatz.num_parameters)


vqe = VQE(noiseless_estimator,ansatz=ansatz, optimizer=optimizer ,
initial_point=initial_point)


In [15]:
result = vqe.compute_minimum_eigenvalue(operator)
print("Result:", result)
print("Optimal parameters:", result.optimal_point)
print(result)

Result: {   'aux_operators_evaluated': None,
    'cost_function_evals': 211,
    'eigenvalue': -3.0,
    'optimal_circuit': <qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 object at 0x7f3854489ae0>,
    'optimal_parameters': {   ParameterVectorElement(θ[1]): 3.2145248780837368,
                              ParameterVectorElement(θ[0]): -0.2858047265893224,
                              ParameterVectorElement(θ[3]): 1.1962640982268984,
                              ParameterVectorElement(θ[2]): 3.5082078175598466,
                              ParameterVectorElement(θ[4]): 1.4891860611364773,
                              ParameterVectorElement(θ[5]): -0.27443975419356503,
                              ParameterVectorElement(θ[6]): 0.9817396780463605,
                              ParameterVectorElement(θ[7]): 0.9153507446594463,
                              ParameterVectorElement(θ[8]): 3.0647402766532297,
                              ParameterVectorElement(θ[9]): 0.25119

In [16]:
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

optimal_points=result.optimal_parameters
param_values = [value for value in optimal_points.values()]

ansatz = EfficientSU2(num_qubits=5, reps=1, entanglement="linear", insert_barriers=True)


parameters = param_values # Por ejemplo, una lista de valores para los parámetros

# Asigna los valores de los parámetros al ansatz
bound_ansatz = ansatz.assign_parameters(parameters)

# Puedes obtener el circuito resultante con los valores asignados
bound_circuit = bound_ansatz

# Dibujar el circuito resultante
qc2=QuantumCircuit(5,5)
qc2=bound_circuit.decompose()
sv2 = Statevector(qc2)
sv2.draw(output='latex', prefix='Estado=')
#qc2.measure_all()
#qc2.draw('mpl')

<IPython.core.display.Latex object>

In [17]:
print("El valor esperado para el estado |010> es:", sv2.expectation_value(operator))

El valor esperado para el estado |010> es: (-2.9889278344590755+0j)


In [10]:
from qiskit_algorithms import NumPyMinimumEigensolver
solver2 = NumPyMinimumEigensolver()
result2 = solver2.compute_minimum_eigenvalue(operator)
print(result2)
result2.eigenstate.draw(output='latex', prefix='Estado de mínima energía=')

{   'aux_operators_evaluated': None,
    'eigenstate': Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2)),
    'eigenvalue': (-3+0j)}


<IPython.core.display.Latex object>

In [18]:
print("El valor esperado para el estado |010> es:", result2.eigenstate.expectation_value(operator))

El valor esperado para el estado |010> es: (-3+0j)
